In [1]:
# Import the modules
import os
import pandas as pd
from IPython.display import display

import MERFISH_probe_design.IO.file_io as fio
import MERFISH_probe_design.probe_design.probe_dict as p_d
import MERFISH_probe_design.probe_design.OTTable_dict as ot
import MERFISH_probe_design.probe_design.readout_sequences as rs
import MERFISH_probe_design.probe_design.probe_selection as ps
import MERFISH_probe_design.probe_design.quality_check as qc
from MERFISH_probe_design.probe_design import filters
from MERFISH_probe_design.probe_design import plot
from MERFISH_probe_design.probe_design import primer_design

In [2]:
# Define all the input files you need in this script
transcripts_fasta_file = 'temporary_data/gencode.v49.transcripts_plus_tRNA.fa' 

In [3]:
# Load the transcriptome
transcriptome = fio.load_transcriptome(transcripts_fasta_file, None, style='gencode')
transcriptome

Loaded 534391 transcripts.
return gencode style transcriptome
{'transcript_id': 'ENST00000832824.1', 'gene_id': 'ENSG00000290825.2', 'havana_gene': '-', 'havana_transcript': '-', 'transcript_name': 'DDX11L16-260', 'gene_name': 'DDX11L16', 'length': '1379', 'transcript_type': 'lncRNA'}


,transcript_id,gene_id,havana_gene,havana_transcript,transcript_name,gene_short_name,length,transcript_type,sequence,FPKM
0,ENST00000832824.1,ENSG00000290825.2,-,-,DDX11L16-260,DDX11L16,1379,lncRNA,CGCAGAGACGGGTAGAACCTCAGTAATCCGAAAAGCCGGGATCGAC...,1
1,ENST00000832825.1,ENSG00000290825.2,-,-,DDX11L16-261,DDX11L16,1417,lncRNA,GAGACGGGTAGAACCTCAGTAATCCGAAAAGCCGGGATCGACCGCC...,1
2,ENST00000832826.1,ENSG00000290825.2,-,-,DDX11L16-262,DDX11L16,1538,lncRNA,CCTGCTGGCAGCTGGGGACACTGCCGGGCCCTCTTGCTCCAACAGT...,1
3,ENST00000832827.1,ENSG00000290825.2,-,-,DDX11L16-263,DDX11L16,1519,lncRNA,CTGCTGGCAGCTGGGGACACTGCCGGGCCCTCTTGCTCCAACAGTA...,1
4,ENST00000832828.1,ENSG00000290825.2,-,-,DDX11L16-264,DDX11L16,1762,lncRNA,GACACTGCCGGGCCCTCTTGCTCCAACAGTACTGGCGGATTATAGG...,1
...,...,...,...,...,...,...,...,...,...,...
534386,TRNST00020088.1,TRNSG00020088.1,-,-,20088,20088,75,tRNA,TGTTAATGAGAGGAGTTGAACCTCTGATTATAAAGTTTTAAGTCTT...,1
534387,TRNST00035969.1,TRNSG00035969.1,-,-,35969,35969,75,tRNA,CTCTAGAATATAGGAATTGAACCTGCACCTGAGAATCCAAAATTCT...,1
534388,TRNST00026266.1,TRNSG00026266.1,-,-,26266,26266,75,tRNA,GTTAAGATGGCAGAGCCCGGTAATCGCATAAAACTTAAAACTTTAC...,1
534389,TRNST00032170.1,TRNSG00032170.1,-,-,32170,32170,74,tRNA,CGTCCTTGGGTGGTCTCGAACCACCAACCTTCCGGTTAACAGCCGA...,1


In [4]:
# Initialize the off-target counting tables
overwrite_ottable = True
# OTTable for rRNA/tRNAs
if 'ottable_rtRNAs' not in locals() or overwrite_ottable:
    #ncRNAs =  fio.load_fasta_into_df(ncRNA_fasta_file)
    ottable_rtRNAs = ot.get_OTTable_for_rtRNAs(transcriptome, 15, search_column='transcript_type')

Found 746 rRNAs/tRNAs from 534391 non-coding RNAs.


In [5]:
# OTTable for the transcriptome.
if 'ottable_transcriptome' not in locals() or overwrite_ottable:
    ottable_transcriptome = ot.get_OTTable_for_transcriptome(transcriptome, 17)

Construct a OTTable using 534391/534391 transcripts with FPKM > 0.
Processed 100000/534391 sequences.
Processed 200000/534391 sequences.
Processed 300000/534391 sequences.
Processed 400000/534391 sequences.
Processed 500000/534391 sequences.


In [6]:
from Bio.Seq import reverse_complement

# load readouts
raw_readouts_df = pd.read_csv('temporary_data/R_readouts_updated.csv')
readouts_df = pd.DataFrame({
    'id' : [s.split('__')[1] for s in raw_readouts_df['Name']],
    'readout_seq' : raw_readouts_df['Sequence (RC in primary probe)'].tolist(),
    'readout_seq_rc' : [reverse_complement(s) for s in raw_readouts_df['Sequence (RC in primary probe)'].tolist()]
}).set_index('id')

readouts_df

,readout_seq,readout_seq_rc
id,,
NDB784,TAGAAATATCCGAGTGGCAT,ATGCCACTCGGATATTTCTA
NDB755,TGCGACATGTTATGCTAGGC,GCCTAGCATAACATGTCGCA
NDB759,TGACTGTCGTTGACTCTCGG,CCGAGAGTCAACGACAGTCA
NDB826,ATATGTCGATGTCCTTAGAC,GTCTAAGGACATCGACATAT
NDB578,AGGCGATACATGAGAGGCGT,ACGCCTCTCATGTATCGCCT
...,...,...
NDB941,GCAATAGCGGCACCTTTCAC,GTGAAAGGTGCCGCTATTGC
NDB846,GGCTATTAGTCGGTCTGATA,TATCAGACCGACTAATAGCC
NDB775,GTTACTCGACCCGCGTGCAA,TTGCACGCGGGTCGAGTAAC


In [7]:
adaptor_df = pd.read_csv('temporary_data/NDB_adaptors.csv', index_col=0, header=None)
adaptor_df.rename(columns={1: 'sequence'}, inplace=True)
adaptor_df['color_id'] = [i.split('_')[-1] for i in adaptor_df.index.tolist()]
adaptor_df['color'] = adaptor_df['color_id'].map({
    '82rc': '750nm',
    '1rc': '647nm',
    '79rc': '565nm',
})

adaptor_color_map = {
    'NDB' + i.split('_')[1]: row['color'] for i, row in adaptor_df.iterrows()
}

adaptor_seq_map = {
    'NDB' + i.split('_')[1]: row['sequence'] for i, row in adaptor_df.iterrows()
}
adaptor_df

,sequence,2,3,color_id,color
0,,,,,
NDB_1_2xStv_82rc,GTTGACGGACTAATCCTGACTTTGCACTGCCGTCCTTGACTTTGCA...,25nm,STD,82rc,750nm
NDB_2_2xStv_1rc,GTTCAGACGCCGTTAAGCGCGATCCGATTGGAACCGTCCCGATCCG...,25nm,STD,1rc,647nm
NDB_3_2xStv_79rc,GTGTGAGGCGCTAGAGCATATGCGAACTGTCCGGCTTTCATGCGAA...,25nm,STD,79rc,565nm
NDB_4_2xStv_82rc,GGCACGCCTAATCTGGACACTTTGCACTGCCGTCCTTGACTTTGCA...,25nm,STD,82rc,750nm
NDB_5_2xStv_1rc,TACGAGCGCTTGGATCCCGTGATCCGATTGGAACCGTCCCGATCCG...,25nm,STD,1rc,647nm
...,...,...,...,...,...
NDB_1279_2xStv_82rc,TAGCGGTGCGTCGGATTCGCTTTGCACTGCCGTCCTTGACTTTGCA...,25nm,STD,82rc,750nm
NDB_1280_2xStv_1rc,CGTGTTGTGGAGACGCGCATGATCCGATTGGAACCGTCCCGATCCG...,25nm,STD,1rc,647nm
NDB_1281_2xStv_79rc,TGGTTAGCCGAACAGGGTCCTGCGAACTGTCCGGCTTTCATGCGAA...,25nm,STD,79rc,565nm


In [8]:
readouts_df['adaptor_color'] = readouts_df.index.map(adaptor_color_map)
readouts_df['adaptor_seq'] = readouts_df.index.map(adaptor_seq_map)
readouts_df['adaptor_seq_rc'] = readouts_df['adaptor_seq'].apply(lambda x: reverse_complement(x))
readouts_df

,readout_seq,readout_seq_rc,adaptor_color,adaptor_seq,adaptor_seq_rc
id,,,,,
NDB784,TAGAAATATCCGAGTGGCAT,ATGCCACTCGGATATTTCTA,750nm,TAGAAATATCCGAGTGGCATTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAATGCCA...
NDB755,TGCGACATGTTATGCTAGGC,GCCTAGCATAACATGTCGCA,647nm,TGCGACATGTTATGCTAGGCGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCGCCTAG...
NDB759,TGACTGTCGTTGACTCTCGG,CCGAGAGTCAACGACAGTCA,565nm,TGACTGTCGTTGACTCTCGGTGCGAACTGTCCGGCTTTCATGCGAA...,TGAAAGCCGGACAGTTCGCATGAAAGCCGGACAGTTCGCACCGAGA...
NDB826,ATATGTCGATGTCCTTAGAC,GTCTAAGGACATCGACATAT,750nm,ATATGTCGATGTCCTTAGACTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGTCTAA...
NDB578,AGGCGATACATGAGAGGCGT,ACGCCTCTCATGTATCGCCT,647nm,AGGCGATACATGAGAGGCGTGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCACGCCT...
...,...,...,...,...,...
NDB941,GCAATAGCGGCACCTTTCAC,GTGAAAGGTGCCGCTATTGC,647nm,GTGAAAGGTGCCGCTATTGCGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCGCAATA...
NDB846,GGCTATTAGTCGGTCTGATA,TATCAGACCGACTAATAGCC,565nm,TATCAGACCGACTAATAGCCTGCGAACTGTCCGGCTTTCATGCGAA...,TGAAAGCCGGACAGTTCGCATGAAAGCCGGACAGTTCGCAGGCTAT...
NDB775,GTTACTCGACCCGCGTGCAA,TTGCACGCGGGTCGAGTAAC,750nm,TTGCACGCGGGTCGAGTAACTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGTTACT...


In [9]:
# Compute the primer off-targets to rRNA/tRNAs and the transcriptome
ot.calc_OTs_df(readouts_df, ottable_rtRNAs, 'adaptor_seq_rc', 'adaptor_seq_rc_OT_rtRNA', 15)
ot.calc_OTs_df(readouts_df, ottable_transcriptome, 'adaptor_seq_rc', 'adaptor_seq_rc_OT_transcriptome', 17)

ot.calc_OTs_df(readouts_df, ottable_rtRNAs, 'adaptor_seq', 'adaptor_seq_OT_rtRNA', 15)
ot.calc_OTs_df(readouts_df, ottable_transcriptome, 'adaptor_seq', 'adaptor_seq_OT_transcriptome', 17)
readouts_df.sort_values(by=['adaptor_seq_rc_OT_rtRNA', 'adaptor_seq_rc_OT_transcriptome', 'adaptor_seq_OT_rtRNA', 'adaptor_seq_OT_transcriptome'], ascending=False)[:20]

,readout_seq,readout_seq_rc,adaptor_color,adaptor_seq,adaptor_seq_rc,adaptor_seq_rc_OT_rtRNA,adaptor_seq_rc_OT_transcriptome,adaptor_seq_OT_rtRNA,adaptor_seq_OT_transcriptome
id,,,,,,,,,
NDB385,GGCACAACACTAGCGTAGTC,GACTACGCTAGTGTTGTGCC,750nm,GACTACGCTAGTGTTGTGCCTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGGCACA...,0,87,0,11
NDB866,AGTACATCGCTTGCTGCAAA,TTTGCAGCAAGCGATGTACT,647nm,AGTACATCGCTTGCTGCAAAGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCTTTGCA...,0,30,0,0
NDB643,CGAACGACCTGTTACCTTGC,GCAAGGTAACAGGTCGTTCG,750nm,CGAACGACCTGTTACCTTGCTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGCAAGG...,0,11,0,0
NDB616,GCACGTATGTCCCGTCCATT,AATGGACGGGACATACGTGC,750nm,GCACGTATGTCCCGTCCATTTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAAATGGA...,0,11,0,0
NDB694,GGCCAGCGATTAAGTATACC,GGTATACTTAATCGCTGGCC,750nm,GGTATACTTAATCGCTGGCCTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGGCCAG...,0,3,0,0
NDB856,GCTCGCAATCCTAATGTATA,TATACATTAGGATTGCGAGC,750nm,GCTCGCAATCCTAATGTATATTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAATATACA...,0,1,0,1
NDB790,GGTGTCGGTTAAATATTCCT,AGGAATATTTAACCGACACC,750nm,GGTGTCGGTTAAATATTCCTTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAAGGAAT...,0,1,0,0
NDB487,TTCACGAACGTCTCTATCTA,TAGATAGAGACGTTCGTGAA,750nm,TTCACGAACGTCTCTATCTATTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAATAGATA...,0,1,0,0
NDB835,TAGGCATACGTTCGAGTTGA,TCAACTCGAACGTATGCCTA,750nm,TCAACTCGAACGTATGCCTATTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAATAGGCA...,0,1,0,0


In [10]:
readouts_df.to_csv('temporary_data/readout_ot_counts.csv')

In [11]:
# Select readouts with zero off-targets
selected_readouts_df = readouts_df[
    (readouts_df['adaptor_seq_rc_OT_rtRNA'] == 0) &
    (readouts_df['adaptor_seq_rc_OT_transcriptome'] == 0) &
    (readouts_df['adaptor_seq_OT_rtRNA'] == 0) &
    (readouts_df['adaptor_seq_OT_transcriptome'] == 0)
].copy()
selected_readouts_df

,readout_seq,readout_seq_rc,adaptor_color,adaptor_seq,adaptor_seq_rc,adaptor_seq_rc_OT_rtRNA,adaptor_seq_rc_OT_transcriptome,adaptor_seq_OT_rtRNA,adaptor_seq_OT_transcriptome
id,,,,,,,,,
NDB755,TGCGACATGTTATGCTAGGC,GCCTAGCATAACATGTCGCA,647nm,TGCGACATGTTATGCTAGGCGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCGCCTAG...,0,0,0,0
NDB759,TGACTGTCGTTGACTCTCGG,CCGAGAGTCAACGACAGTCA,565nm,TGACTGTCGTTGACTCTCGGTGCGAACTGTCCGGCTTTCATGCGAA...,TGAAAGCCGGACAGTTCGCATGAAAGCCGGACAGTTCGCACCGAGA...,0,0,0,0
NDB826,ATATGTCGATGTCCTTAGAC,GTCTAAGGACATCGACATAT,750nm,ATATGTCGATGTCCTTAGACTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGTCTAA...,0,0,0,0
NDB578,AGGCGATACATGAGAGGCGT,ACGCCTCTCATGTATCGCCT,647nm,AGGCGATACATGAGAGGCGTGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCACGCCT...,0,0,0,0
NDB795,GGGCGTCTGTTACACGTTTG,CAAACGTGTAACAGACGCCC,565nm,GGGCGTCTGTTACACGTTTGTGCGAACTGTCCGGCTTTCATGCGAA...,TGAAAGCCGGACAGTTCGCATGAAAGCCGGACAGTTCGCACAAACG...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
NDB941,GCAATAGCGGCACCTTTCAC,GTGAAAGGTGCCGCTATTGC,647nm,GTGAAAGGTGCCGCTATTGCGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCGCAATA...,0,0,0,0
NDB846,GGCTATTAGTCGGTCTGATA,TATCAGACCGACTAATAGCC,565nm,TATCAGACCGACTAATAGCCTGCGAACTGTCCGGCTTTCATGCGAA...,TGAAAGCCGGACAGTTCGCATGAAAGCCGGACAGTTCGCAGGCTAT...,0,0,0,0
NDB775,GTTACTCGACCCGCGTGCAA,TTGCACGCGGGTCGAGTAAC,750nm,TTGCACGCGGGTCGAGTAACTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGTTACT...,0,0,0,0


In [12]:
rs_750 = selected_readouts_df[selected_readouts_df['adaptor_color'] == '750nm'].index.tolist()
rs_647 = selected_readouts_df[selected_readouts_df['adaptor_color'] == '647nm'].index.tolist()
rs_565 = selected_readouts_df[selected_readouts_df['adaptor_color'] == '565nm'].index.tolist()
n_rounds = max(len(rs_750), len(rs_647), len(rs_565))

selected_ids = []
for i in range(n_rounds):
    if i < len(rs_750):
        selected_ids.append(rs_750[i])
    if i < len(rs_647):
        selected_ids.append(rs_647[i])
    if i < len(rs_565):
        selected_ids.append(rs_565[i])

selected_readouts_df = selected_readouts_df.loc[selected_ids]
selected_readouts_df[['readout_seq', 'readout_seq_rc', 'adaptor_color', 'adaptor_seq']].to_csv('temporary_data/selected_readouts.csv')
selected_readouts_df

,readout_seq,readout_seq_rc,adaptor_color,adaptor_seq,adaptor_seq_rc,adaptor_seq_rc_OT_rtRNA,adaptor_seq_rc_OT_transcriptome,adaptor_seq_OT_rtRNA,adaptor_seq_OT_transcriptome
id,,,,,,,,,
NDB826,ATATGTCGATGTCCTTAGAC,GTCTAAGGACATCGACATAT,750nm,ATATGTCGATGTCCTTAGACTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAGTCTAA...,0,0,0,0
NDB755,TGCGACATGTTATGCTAGGC,GCCTAGCATAACATGTCGCA,647nm,TGCGACATGTTATGCTAGGCGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCGCCTAG...,0,0,0,0
NDB759,TGACTGTCGTTGACTCTCGG,CCGAGAGTCAACGACAGTCA,565nm,TGACTGTCGTTGACTCTCGGTGCGAACTGTCCGGCTTTCATGCGAA...,TGAAAGCCGGACAGTTCGCATGAAAGCCGGACAGTTCGCACCGAGA...,0,0,0,0
NDB376,GACTTTGTGGGTACGTTGAT,ATCAACGTACCCACAAAGTC,750nm,GACTTTGTGGGTACGTTGATTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAATCAAC...,0,0,0,0
NDB578,AGGCGATACATGAGAGGCGT,ACGCCTCTCATGTATCGCCT,647nm,AGGCGATACATGAGAGGCGTGATCCGATTGGAACCGTCCCGATCCG...,GGGACGGTTCCAATCGGATCGGGACGGTTCCAATCGGATCACGCCT...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
NDB934,ACTTCCCAAATGCGCGCGAA,TTCGCGCGCATTTGGGAAGT,750nm,TTCGCGCGCATTTGGGAAGTTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAACTTCC...,0,0,0,0
NDB481,ATATACGCAGCGAGGCACCT,AGGTGCCTCGCTGCGTATAT,750nm,AGGTGCCTCGCTGCGTATATTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAATATAC...,0,0,0,0
NDB910,ACTGGTGAACTCCGGAGCGC,GCGCTCCGGAGTTCACCAGT,750nm,GCGCTCCGGAGTTCACCAGTTTTGCACTGCCGTCCTTGACTTTGCA...,GTCAAGGACGGCAGTGCAAAGTCAAGGACGGCAGTGCAAAACTGGT...,0,0,0,0
